In [2]:
# import spot_writer_utils as swu
# #이미지 캡셔닝
import tensorflow as tf
from tensorflow import keras
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

#한글 깨짐 방지
# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'
%matplotlib inline
#단어 추출
from konlpy.tag import *

#단어 유사도 측정
from gensim import models

#kogpt2 문장 생성
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from kogpt2.utils import get_tokenizer
from kogpt2.utils import download, tokenizer
from model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from util.data import NovelDataset
import gluonnlp
import sampling
import kss


/home/piai/anaconda3/envs/pytorch/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [3]:
#캡셔닝
# Find the maximum length of any caption in our dataset
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x
    
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

def calc_max_length(tensor):
    return max(len(t) for t in tensor)

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

def evaluate(image):
    attention_plot = np.zeros((max_length, 64))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(10):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

# 이미지 캡셔닝

In [4]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

image_folder = '/train2014/'
annotation_file = './MSCOCO_train_Korean.json'
PATH = os.path.abspath('.') + image_folder

with open(annotation_file, 'r') as f:
    annotations = json.load(f)

#한글 데이터에 맞게 변형
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for ele in annotations:
  caption = f"<start> {ele['caption_ko']} <end>" #{idx['caption_ko']}
  image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (ele['id'])
  image_path_to_caption[image_path].append(caption)

image_paths = list(image_path_to_caption.keys())
random.shuffle(image_paths)

# Select the first 6000 image_paths from the shuffled set.
# Approximately each image id has 5 captions associated with it, so that will 
# lead to 30,000 examples.
train_image_paths = image_paths[:80000]

train_captions = []
img_name_vector = []

for image_path in train_image_paths:
  caption_list = image_path_to_caption[image_path]
  train_captions.extend(caption_list)
  img_name_vector.extend([image_path] * len(caption_list))

# Choose the top 5000 words from the vocabulary
top_k = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
top_k=10000
vocab_size = top_k + 1
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

#weights 불러오기
encoder.load_weights('./checkpoint_test/encoder')
decoder.load_weights('./checkpoint_test/decoder')

In [5]:
# 이미지의 url 주소를 통해 테스트
# image_url ='https://www.google.com/url?sa=i&url=https%3A%2F%2Fm.bobaedream.co.kr%2Fboard%2Fbbs_view%2Fbest%2F44201&psig=AOvVaw1LZkdivowjBvJKoUlw8qoJ&ust=1608963054772000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCPD17dK86O0CFQAAAAAdAAAAABAP'
# image_extension = image_url[-4:]
# image_path = tf.keras.utils.get_file('image'+image_extension,
#                                      origin=image_url)
# img, image_path=load_image(image_path)
# # 파일에 있는 이미지 파일을 테스트
image_path = 'cafe.jpg'
result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))# 이것도 단어들 합쳐서 문장 만드는 건데 실제 과정에서 필요없음 result만 필요
# ' '.join(result)
# plot_attention(image_path, result, attention_plot) # 이건 과정에서는 필요없음
# opening the image
sentence=' '.join(result) #문장으로 합치기
Image.open(image_path) #이미지 보여주기인데 실제 과정에서는 필요없음

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

# 단어 추출

In [2]:
# hannanum = Hannanum()
kkma = Kkma()
# komoran = Komoran() #코모란 이용
results=kkma.nouns(sentence) #문장에서 명사만 추출 #TF-IDF사용하려고 했으나 한 문장밖에 안되서 의미가 없음
# results1=hannanum.nouns(sentence)
# results2=komoran.nouns(sentence)
# results= list(set(results))#set으로 만들었다가 다시 리스트로 > 중복제거용 > 이게 필요한지 아니면 그냥 중복되도 있는게 좋은지 토의 필요 > 많이 나왔다 > 그만큼 핵심이다.?
print(results)

NameError: name 'Kkma' is not defined

# word embedding (추상 단어 포함시키기)

In [18]:
# 한번 실행후 다시 로드 할필요없으므로 한번만 실행되도록 하는 장치 필요
try:
    if ko_model:#ko_model이 선언되어있으면 pass
        pass
except: #ko_model 없으면 불러오기
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [19]:
#두개의 단어 집합
# results# = ['비행기','하늘','구름'] #단어 추출 결과 리스트에 해당
abstract_dic=['일상', '셀카', '육아', '여행', '웨딩', '커플', '맛집', '운동', '선물', '먹방', '사랑해', '친구', '감성', '힐링', '행복','봄','여름','가을','겨울','그림','휴가','취미','뷰티','점심','쇼핑','생일','감성사진','추억'
] #우리가 지정할 추상 단어 세트

In [20]:
#평균값으로 caption집합과 추상단어간의 유사도 구하기
mean_sim=[]
for ab_word in abstract_dic:
    sim_lst=[]
    for cap_word in results:
        sim_lst.append(ko_model.wv.similarity(cap_word,ab_word))
    mean_sim.append(np.mean(sim_lst))
max_index=mean_sim.index(max(mean_sim))
#단어와 유사도를 묶어서 리스트 만들기
idx_word=[]
for i in range(len(abstract_dic)):
    idx_word.append((mean_sim[i],abstract_dic[i]))
idx_word.sort(reverse=True) #유사도가 높은 순으로 정렬

topn=1 #높은 유사도 topn개 추출
#추출된 단어 기존 단어 리스트에 포함
for i in range(topn):
    results.append(idx_word[i][1])
results=list(set(results))#문장 생성전에는 중복 제거
results

['기차', '식당', '친구', '여자', '식탁', '사람', '유리']

# 문장 생성

In [23]:
ctx= 'cuda:1'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용 #cuda: 1 두번쨰 gpu로 하니까 오류 해결(용량 부족)
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
save_path = '/home/piai/AI project/NarrativeKoGPT2/narrativeKoGPT2/checkpoint/'
load_path = '/home/piai/AI project/NarrativeKoGPT2/narrativeKoGPT2/checkpoint/narrativeKoGPT2_checkpoint.tar'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

In [26]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

using cached model
using cached model


In [27]:
# Device 설정
device = torch.device(ctx)
# 저장한 Checkpoint 불러오기
checkpoint = torch.load(load_path, map_location=device)

# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval() #테스트 모드
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')


In [28]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
tok = SentencepieceTokenizer(tok_path)

using cached model


In [36]:
sent = '친구'

toked = tok(sent)
count = 0
output_size = 10 # 출력하고자 하는 토큰 갯수

while 1:
  input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
  predicts = model(input_ids)
  pred = predicts[0]

  last_pred = pred.squeeze()[-1]
  # top_p 샘플링 방법
  # sampling.py를 통해 random, top-k, top-p 선택 가능.
  gen = sampling.top_p(last_pred, vocab, 0.4)#(last_pred, vocab, 5)#(last_pred, vocab, 0.9)top_p
  # gen = sampling.top_k(last_pred, vocab, 5)

  if count>output_size:
    sent += gen.replace('▁', ' ')
    toked = tok(sent)
    count =0
    break
  sent += gen.replace('▁', ' ')
  toked = tok(sent)
  count += 1
print(sent)
for s in kss.split_sentences(sent):
    print(s)

selected token: 랑 softmax value:tensor(0.4249, grad_fn=<SelectBackward>)
selected token: ▁함께 softmax value:tensor(0.7075, grad_fn=<SelectBackward>)
selected token: 해서 softmax value:tensor(0.9999, grad_fn=<SelectBackward>)
selected token: ▁좋은 softmax value:tensor(0.9999, grad_fn=<SelectBackward>)
selected token: ▁우리 softmax value:tensor(0.9822, grad_fn=<SelectBackward>)
selected token: 딸 softmax value:tensor(1.0000, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.4816, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.6351, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.5753, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.5616, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.5179, grad_fn=<SelectBackward>)
selected token: </s> softmax value:tensor(0.4849, grad_fn=<SelectBackward>)
친구랑 함께해서 좋은 우리딸</s></s></s></s></s></s>
친구랑 함께해서 좋은 우리딸</s></s></s></s></s></s>


In [116]:
sent

'반려묘에 있던    고양이 한 마리에가,</s>'

In [ ]:
#공백 한칸으로 바꾸기 and 특수문자 날리기 필요